In [2]:
import os
from pathlib import Path
from PIL import Image


import torch
from torchvision import transforms
from torch.autograd import Variable

from tqdm import tqdm_notebook as tqdm

import sys
sys.path.insert(0, './model')

import model.classifier as classifier

In [3]:
# Define a global transformer to appropriately scale images and subsequently convert them to a Tensor.
img_size = 224
loader = transforms.Compose([
  transforms.Resize(img_size),
  transforms.CenterCrop(img_size),
  transforms.ToTensor(),
]) 

def load_image(filename):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    image = Image.open(filename).convert('RGB')
    image_tensor = loader(image).float()
    image_var = Variable(image_tensor).unsqueeze(0)
    return image_var

load_image('data/train_data/Drew_Barrymore/aligned_vgg_bfff48d47d2ec61e678fa23f885df73d.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_data/Drew_Barrymore/aligned_vgg_bfff48d47d2ec61e678fa23f885df73d.jpg'

In [8]:
# Load Training Data

train_data_folder = './data/train_data'

class_to_name = [os.path.basename(f.path) for f in os.scandir(train_data_folder) if f.is_dir()]

train_id_to_file = {i : path 
                    for (i,path) in enumerate(Path(train_data_folder).glob("*/*.jpg"))}

train_id_to_class = {i : class_to_name.index(os.path.basename(os.path.dirname(str(path))))
                     for (i,path) in enumerate(Path(train_data_folder).glob("*/*.jpg"))}

train_ids = list(train_id_to_file.keys())

# Load Validation Data

val_data_folder = './data/eval_data'

val_id_to_file = {i : path 
                    for (i,path) in enumerate(Path(val_data_folder).glob("*/*.jpg"))}

val_id_to_class = {i : class_to_name.index(os.path.basename(os.path.dirname(str(path))))
                     for (i,path) in enumerate(Path(val_data_folder).glob("*/*.jpg"))}

val_ids = list(val_id_to_file.keys())


print("Classes:")
print(class_to_name)

print("\nTraining Set Size: %s" % len(train_ids))
print("\nValidation Set Size: %s" % len(val_ids))

print("\nSample Images:")
print(train_id_to_file[len(train_ids)-1])
print(class_to_name[train_id_to_class[len(train_ids)-1]])

load_image(train_id_to_file[len(train_ids)-1])

print(val_id_to_file[0])
print(class_to_name[val_id_to_class[0]])

load_image(val_id_to_file[0])


Classes:
['Zac_Efron', 'Cristiano_Ronaldo', 'Aaron_Eckhart', 'Clive_Owen', 'Alyssa_Milano', 'Christina_Ricci', 'Nicole_Richie', 'Brad_Pitt', 'Julia_Roberts', 'Drew_Barrymore']

Training Set Size: 1364

Validation Set Size: 100

Sample Images:
data/train_data/Drew_Barrymore/aligned_vgg_bfff48d47d2ec61e678fa23f885df73d.jpg
Drew_Barrymore
data/eval_data/Zac_Efron/aligned_vgg_0c5e844bc8df7d4742c0a6f790a7e6f1 4.46.24 PM.jpg
Zac_Efron


tensor([[[[0.4353, 0.4431, 0.4353,  ..., 0.0275, 0.0314, 0.0353],
          [0.4314, 0.4392, 0.4353,  ..., 0.0314, 0.0392, 0.0431],
          [0.4275, 0.4392, 0.4392,  ..., 0.0392, 0.0471, 0.0510],
          ...,
          [0.0431, 0.0471, 0.0510,  ..., 0.0157, 0.0157, 0.0157],
          [0.0314, 0.0353, 0.0392,  ..., 0.0157, 0.0157, 0.0157],
          [0.0275, 0.0314, 0.0353,  ..., 0.0157, 0.0157, 0.0157]],

         [[0.4118, 0.4196, 0.4118,  ..., 0.0275, 0.0314, 0.0353],
          [0.4078, 0.4157, 0.4118,  ..., 0.0314, 0.0392, 0.0431],
          [0.4039, 0.4157, 0.4157,  ..., 0.0392, 0.0471, 0.0510],
          ...,
          [0.0510, 0.0549, 0.0549,  ..., 0.0157, 0.0157, 0.0157],
          [0.0510, 0.0549, 0.0549,  ..., 0.0157, 0.0157, 0.0157],
          [0.0471, 0.0510, 0.0549,  ..., 0.0157, 0.0157, 0.0157]],

         [[0.2863, 0.2941, 0.2863,  ..., 0.0353, 0.0392, 0.0431],
          [0.2824, 0.2902, 0.2863,  ..., 0.0392, 0.0471, 0.0510],
          [0.2784, 0.2902, 0.2902,  ..., 0

In [9]:
model = classifier.FaceClassifier()

if torch.cuda.is_available():
    model = model.cuda()
    
print(model)

FaceClassifier(
  (model): Vgg_face_dag(
    (conv1_1): Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu1_1): ReLU(inplace)
    (conv1_2): Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu1_2): ReLU(inplace)
    (pool1): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu2_1): ReLU(inplace)
    (conv2_2): Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu2_2): ReLU(inplace)
    (pool2): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu3_1): ReLU(inplace)
    (conv3_2): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu3_2): ReLU(inplace)
    (conv3_3): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu3_3):

In [ ]:
# TODO - Fine-tuning

# Load Training Images and Labels

print('Loading training images...')

#train_ids = train_ids[:50]

images = Variable(torch.zeros((len(train_ids),3,img_size,img_size)))
labels = Variable(torch.zeros(len(train_ids)).long())

for i,train_id in enumerate(tqdm(train_ids)):
    # Prepare the image tensors
    images[i] = load_image(train_id_to_file[train_id])
    
    # Prepare the labels
    labels[i] = train_id_to_class[train_id]

# Load Validation Images and Labels

print('Loading validation images...')

val_images = Variable(torch.zeros((len(val_ids),3,img_size,img_size)))
val_labels = Variable(torch.zeros(len(val_ids)).long())

for i,val_id in enumerate(tqdm(val_ids)):
    # Prepare the image tensors
    val_images[i] = load_image(val_id_to_file[val_id])
    
    # Prepare the labels
    val_labels[i] = val_id_to_class[val_id]
    
X = images
y = labels

X_val = val_images
y_val = val_labels

print('Fine-tuning the model...')

model.tune(X, y, X_val, y_val)

print('Saving the model...')

filename = 'model/tuned_classifier.pth'
torch.save(model.state_dict(), filename)

print('Model saved as %s' % filename)

Loading training images...



Loading validation images...



Fine-tuning the model...


[1,     5] loss: 2.278
[1,    10] loss: 2.222
[1,    15] loss: 2.161
[1,    20] loss: 2.104
